## Random Forest - Score: 0.87874

Validation set not used in final model training.

In [1]:
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_auc_score

pd.options.display.max_rows = 20
pd.options.display.max_columns = 100

### Process Data

In [2]:
# define path to data
path = URLs.path('march-comp')

In [3]:
# define dependent var
dep_var = 'target'

In [4]:
# load tabularpandas object
to = load_pickle(path/'march_tabular.pkl')

In [5]:
# get preds and targets for train and valid set
xs,y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y

### Create Random Forest

In [21]:
# create function to call sklearn random forest regressor
def rf(xs, y, n_estimators=200, max_features=0.5, min_samples_leaf=5, **kwargs):
    return RandomForestRegressor(n_jobs=-1, n_estimators=n_estimators, max_features=max_features,
        min_samples_leaf=min_samples_leaf, oob_score=True).fit(xs, y)

In [22]:
# fit random forest to train set
m = rf(xs, y);

In [23]:
# get performance
roc_auc_score(y, m.predict(xs)), roc_auc_score(valid_y, m.predict(valid_xs))

(0.988897443049643, 0.8831245062623054)

### Submit Predictions

In [24]:
# read in test set
test = pd.read_csv(path/'test.csv', low_memory=False)

In [25]:
# convert test set to fastai tabular dataloader
dls = to.dataloaders(bs=1024)
test_dl = dls.test_dl(test, bs=1024)

In [26]:
# checking length of test set & dataloader - make sure same length
len(test), len(test_dl.train.xs)

(200000, 200000)

In [27]:
# get preds
sub_y = m.predict(test_dl.train.xs)

In [28]:
# create submission df
submission = pd.DataFrame({"id": test.id,"target": sub_y})

In [29]:
# save submission to csv
submission.to_csv('RF001.csv', index=False)